In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

tokenizer = AutoTokenizer.from_pretrained('mesolitica/mistral-7b-4096-fpf')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type='nf4',
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/mistral-7b-4096-fpf', 
    quantization_config=nf4_config,
    use_flash_attention_2=True
)

[2023-10-21 13:25:10,069] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-10-21 13:25:10.415860: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-21 13:25:11.085951: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
import time
from tqdm import tqdm

kwargs = {
    'temperature': 0.9, 
    'max_new_tokens': 512, 
    'top_p': 0.95,
    'repetition_penalty': 1.1, 
    'do_sample': True,
    'num_beams': 1,
}

In [21]:
inputs = tokenizer(['Manusia: macam mana nak code websocket guna htmx? Bot:'], return_tensors='pt').to('cuda')
generate_kwargs = dict(inputs)
generate_kwargs = {**generate_kwargs, **kwargs}

In [22]:
o = model.generate(**generate_kwargs)
tokenizer.decode(o[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> Manusia: macam mana nak code websocket guna htmx? Bot: Hanya dengan menggunakan JavaScript dan HTML, anda boleh menghantar data dari bot ke pelayan menggunakan WebSocket. Berikut adalah contoh kod untuk melaksanakannya: ``` var ws = new WebSocket.WebSocket("ws://your-website/"); ws.onopen = function() { // Menyambung ke pelayan }; ws.onmessage = function(event) { // Menerima data dari pelayan }; ws.onclose = function() { // Menutup sambungan }; var message = "Hello World!"; // Mesej untuk dihantar ke pelayan ws.send(message); ``` Dalam kod ini, kami membuat objek `WebSocket` baru menggunakan URL pelayan WebSocket. Kemudian, kami menentukan pendengar acara untuk acara `open`, `message`, dan `close`. Fungsi pendengar `open` dipicu apabila sambungan ke pelayan berjaya. Dalam fungsi ini, anda boleh menetapkan WebSocket supaya dapat menerima mesej yang masuk. Fungsi pendengar `message` dipicu apabila mesej diterima dari pelayan. Anda boleh menerima mesej yang masuk dan menguraikan data

In [24]:
inputs = tokenizer(['Manusia: macam mana nak code sentiment guna malaya? Bot:'], return_tensors='pt').to('cuda')
generate_kwargs = dict(inputs)
generate_kwargs = {**generate_kwargs, **kwargs}
o = model.generate(**generate_kwargs)
tokenizer.decode(o[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> Manusia: macam mana nak code sentiment guna malaya? Bot: Dengan menggunakan API Pengesanan Emosi, kita boleh melatih model pembelajaran mesin untuk mengenali emosi dalam teks. Dalam bahasa pengaturcaraan Python, ini boleh dicapai dengan menggunakan perpustakaan Natural Language Processing (NLP) seperti NLTK dan Stanford NER. Berikut adalah contoh bagaimana untuk memproses data sentimen menggunakan algoritma NLTK: ```python import nltk from nltk.sentiment import SentimentIntensityAnalyzer # muat turun leksikon yang diperlukan untuk analisis sentimen nltk.download(\'vader_lexicon\') sid = SentimentIntensityAnalyzer() # annotate rentetan "Saya suka produk ini" data_sentimen = sid.polarity_scores("Saya suka produk ini") # output: {\'neg\': 0.0, \'neu\': 0.5, \'pos\': 0.5, \'compound\': 1.0} ``` Untuk menambahkan keupayaan manusia dalam proses ini, kita boleh melatih model pembelajaran mesin menggunakan senarai sentimen yang diberikan oleh manusia. Setelah model dilatih, ia boleh meram

In [25]:
print(tokenizer.decode(o[0]))

<s> Manusia: macam mana nak code sentiment guna malaya? Bot: Dengan menggunakan API Pengesanan Emosi, kita boleh melatih model pembelajaran mesin untuk mengenali emosi dalam teks. Dalam bahasa pengaturcaraan Python, ini boleh dicapai dengan menggunakan perpustakaan Natural Language Processing (NLP) seperti NLTK dan Stanford NER. Berikut adalah contoh bagaimana untuk memproses data sentimen menggunakan algoritma NLTK: ```python import nltk from nltk.sentiment import SentimentIntensityAnalyzer # muat turun leksikon yang diperlukan untuk analisis sentimen nltk.download('vader_lexicon') sid = SentimentIntensityAnalyzer() # annotate rentetan "Saya suka produk ini" data_sentimen = sid.polarity_scores("Saya suka produk ini") # output: {'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound': 1.0} ``` Untuk menambahkan keupayaan manusia dalam proses ini, kita boleh melatih model pembelajaran mesin menggunakan senarai sentimen yang diberikan oleh manusia. Setelah model dilatih, ia boleh meramalkan senti